# COGS 108 - Data Checkpoint

# Names

- Arnav Tayal
- Chaewon Heo
- Martha Chow
- Zhiyi Zhu

<a id='research_question'></a>
# Research Question

Main question: Are characteristics of an individual’s diet (such as the amount of calories, macronutrients, and/or micronutrients) correlated with their likelihood of contracting COVID-19?

Sub-questions: Can we correlate diet type (eg. vegetarian, keto, pescetarian) with an individual’s likelihood of contracting COVID-19? Which characteristic is the strongest predictor?

Potential extension: Does correlation between a characteristic necessarily mean causation (eg. If we find that there is a positive correlation between kcal/day and likelihood of contracting COVID-19, does that necessarily mean that a high calorie diet leads to higher vulnerability to COVID-19?)? How much literature is there on databases to support the causal relationship?

# Dataset(s)


- Dataset Name: Supply_Quantity_Kg
- Link to the dataset:https://github.com/mariarencode/COVID_19_Dataset_Challenge/tree/master/Food_Supply_Quantity_Kg
- Number of observations: 174

Each file in this dataset records the amount of a particular food category of each country in kilograms/capita/yr

- Dataset Name: Fat Supply
- Link to the dataset: https://github.com/mariarencode/COVID_19_Dataset_Challenge/tree/master/Fat_Supply
- Number of observations: 174

Each file in this dataset records the amount of fat in a particular food category of each country in grams/capita/yr

- Dataset Name: Protein Supply
- Link to the dataset: https://github.com/mariarencode/COVID_19_Dataset_Challenge/tree/master/Protein_Supply
- Number of observations: 174

Each file in this dataset records the amount of protein in a particular food category of each country in grams/capita/yr

We plan to pick particular columns from these datasets and combine them using the merge function in pandas, each row will be unique to a country 

# Setup

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# packages for importing files
import os
import glob

__Get the add the Confirmed,Deaths,Recovered,Active,Population to the dataframe; those are directly extracted from the dataframe that records the precentages. Beside the population, the unit for the other 4 quantitative columns are in percentages.__

NOTE: the cleaned data (the percentages) from github is somehow different from the ones on kaggle. Which one do we want to use for the rates?

In [ ]:
df_covid = pd.read_csv("/cellar/users/zhzhu/projects/random/COGS108/COVID_19_Dataset_Challenge/Cleaned_Datasets/Food_Supply_Quantity_kg_Data.csv",usecols=['Country','Confirmed','Deaths','Recovered','Active','Population'])
df_covid.columns = (map(lambda x: x.lower(), df_covid.columns))
df_covid.head(3)

# Data Cleaning

__Process data for food quantity (unit: kg/capita/yr)__

In [ ]:
# read in most csvs as a list
path = '/cellar/users/zhzhu/projects/random/COGS108/COVID_19_Dataset_Challenge/Food_Supply_Quantity_Kg' # TODO: change this to the path that contains Food_Supply_Quantity_Kg on your enviroment
all_files = glob.glob(os.path.join(path, "*.csv"))

In [ ]:
# function that standardize food name; all lowercase + no space + no special character
def getName(n):
    name = n.replace(' ','_')
    name = name.replace('-','')
    name = name.replace(',','')
    name = name.replace('&','')
    name = name.replace('__','_')
    return name.lower()

In [ ]:
# read in the first csv in the directory so we can merge other csvs with it
col_list = ["Area", "Item", "Value"] # we only need the country and the quantity of food from the file
df_quantity = pd.read_csv(all_files[0], usecols=col_list)
name = getName(df_quantity.loc[0].at["Item"])
df_quantity = df_quantity.drop('Item', 1)
df_quantity.columns = ['country',name]

# merge all csvs in the directory into df_quantity
for i in all_files[1:len(all_files)]:
    # extract the name of the file
    tmp = pd.read_csv(i, usecols=col_list)
    name = getName(tmp.loc[0].at["Item"])
    tmp = tmp.drop('Item', 1)
    tmp.columns = ['country',name] 
    df_quantity = pd.merge(df_quantity,tmp,on ='country', how='outer')

In [ ]:
# we also need the Animal Product and Vegetal Product csvs that are in different forms, so add them to dataframe seperately
animal = pd.read_csv("/cellar/users/zhzhu/projects/random/COGS108/COVID_19_Dataset_Challenge/Unprocessed_Data/FAOSTAT_food_kg_animal.csv") #TODO: change this to the unprocess data directory in your enviroment
animal.columns = ['country','animal_products'] 
vegetal_prod = pd.read_csv("/cellar/users/zhzhu/projects/random/COGS108/COVID_19_Dataset_Challenge/Unprocessed_Data/FAOSTAT_food_kg_vegetal_prod.csv") #TODO: change this to the unprocess data directory in your enviroment
vegetal_prod.columns = ['country','vegetal_products']
df_quantity = pd.merge(df_quantity,animal,on ='country', how='outer')
df_quantity = pd.merge(df_quantity,vegetal_prod,on ='country', how='outer')

In [ ]:
# sort the columns, with country at the first column, and the food columns in alphabetical order
df_quantity = df_quantity.sort_index(axis=1)
first_column = df_quantity.pop('country')
df_quantity.insert(0, 'country', first_column)

# fill na with 0
df_quantity = df_quantity.fillna(0)

In [ ]:
# combine with total quantity dataframe with the covid dataframe
# TODO: if we want the df_covid to be a seperate dataframe, just delete this cell.
df_quantity = pd.merge(df_quantity,df_covid,on ='country', how='right') # join on right since the covid rates are essential for further analysis, so we don't want NA 

In [ ]:
df_quantity.head(6)

In [ ]:
df_quantity.shape

__Process data for fat ( original unit: g/capita/day; will be converted to kg/capita/year)__

In [ ]:
# function that converts g/capita/day to kg/capita/yr 
# TODO: someone check if this is correct
def standUnit(x):
    # g -> kg: / 1000
    # day -> year: * 365
    return x / 1000 * 365

In [ ]:
# read in most csvs as a list
path = '/cellar/users/zhzhu/projects/random/COGS108/COVID_19_Dataset_Challenge/Fat_Supply' # TODO: change this to the path that contains Food_Supply_Quantity_Kg on your enviroment
all_files = glob.glob(os.path.join(path, "*.csv"))

# read in the first csv in the directory so we can merge other csvs with it
col_list = ["Area", "Item", "Value"] # we only need the country and the quantity of food from the file
df_fat = pd.read_csv(all_files[0], usecols=col_list)
name = getName(df_fat.loc[0].at["Item"])
df_fat = df_fat.drop('Item', 1)
df_fat.columns = ['country',name]

# merge all csvs in the directory into df_fat
for i in all_files[1:len(all_files)]:
    # extract the name of the file
    tmp = pd.read_csv(i, usecols=col_list)
    name = getName(tmp.loc[0].at["Item"])
    tmp = tmp.drop('Item', 1)
    tmp.columns = ['country',name] 
    df_fat = pd.merge(df_fat,tmp,on ='country', how='outer')
    
# sort the columns, with country at the first column, and the food columns in alphabetical order
df_fat = df_fat.sort_index(axis=1)
first_column = df_fat.pop('country')
df_fat.insert(0, 'country', first_column)

# replace na with 0
df_fat = df_fat.fillna(0)
df_fat.head(3)

In [ ]:
# convert g/capita/day to kg/capita/yr
df_fat[df_fat.columns[1:len(df_fat.columns)]] = df_fat[df_fat.columns[1:len(df_fat.columns)]].applymap(standUnit)
df_fat.head(3)

In [ ]:
df_fat = pd.merge(df_fat,df_covid,on ='country', how='right') # join on right since the covid rates are essential for further analysis, so we don't want NA in the covid rates column
df_fat.head(3)

__Process protein as fat__

In [ ]:
# read in most csvs as a list
path = '/cellar/users/zhzhu/projects/random/COGS108/COVID_19_Dataset_Challenge/Protein_Supply' # TODO: change this to the path that contains Food_Supply_Quantity_Kg on your enviroment
all_files = glob.glob(os.path.join(path, "*.csv"))

# read in the first csv in the directory so we can merge other csvs with it
col_list = ["Area", "Item", "Value"] # we only need the country and the quantity of food from the file
df_protein = pd.read_csv(all_files[0], usecols=col_list)
name = getName(df_protein.loc[0].at["Item"])
df_protein = df_protein.drop('Item', 1)
df_protein.columns = ['country',name]

# merge all csvs in the directory into df_protein
for i in all_files[1:len(all_files)]:
    # extract the name of the file
    tmp = pd.read_csv(i, usecols=col_list)
    name = getName(tmp.loc[0].at["Item"])
    tmp = tmp.drop('Item', 1)
    tmp.columns = ['country',name] 
    df_protein = pd.merge(df_protein,tmp,on ='country', how='outer')
    
# sort the columns, with country at the first column, and the food columns in alphabetical order
df_protein = df_protein.sort_index(axis=1)
first_column = df_protein.pop('country')
df_protein.insert(0, 'country', first_column)

# replace na with 0
df_protein = df_protein.fillna(0)
df_protein.head(3)

In [ ]:
# convert g/capita/day to kg/capita/yr
df_protein[df_protein.columns[1:len(df_protein.columns)]] = df_protein[df_protein.columns[1:len(df_protein.columns)]].applymap(standUnit)
df_protein.head(3)

In [ ]:
df_protein = pd.merge(df_protein,df_covid,on ='country', how='right') # join on right since the covid rates are essential for further analysis, so we don't want NA in the covid rates column
df_protein.head(3)